### Google Colab utils

In [ ]:
#!pip install keras-tuner

In [ ]:
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [ ]:
#!kill -9 -1

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd "/gdrive/My Drive/air-pollution"

### Modeling

In [1]:
import warnings
warnings.filterwarnings('ignore')

from models import *
from kerastuner.tuners import RandomSearch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/home/zafir/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
train_encoder_input_data = np.load('./data/third-order/Centar/train_encoder_input_data.npy')
train_decoder_input_data = np.load('./data/third-order/Centar/train_decoder_input_data.npy')
train_decoder_target_data = np.load('./data/third-order/Centar/train_decoder_target_data.npy')

valid_encoder_input_data = np.load('./data/third-order/Centar/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load('./data/third-order/Centar/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load('./data/third-order/Centar/valid_decoder_target_data.npy')

In [3]:
print(train_encoder_input_data.shape)
print(train_decoder_input_data.shape)
print(train_decoder_target_data.shape)

(67396, 24, 23)
(67396, 12, 21)
(67396, 12, 2)


In [4]:
Tx, encoder_input_dim = (train_encoder_input_data.shape[1], 
                         train_encoder_input_data.shape[2])
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

# we are predicting the pollution only, leave out the mask
decoder_output_dim = 1  

In [5]:
batch_size = 64
epochs = 250
max_trials = 1500
executions_per_trial = 1
patience = 20

### Standard

In [6]:
model_builder = StandardSeq2Seq(Tx, Ty, encoder_input_dim, 
                             decoder_input_dim, decoder_output_dim)

tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner', 
                     project_name='standard')

In [7]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 67396 samples, validate on 3388 samples
Epoch 1/250
 8640/67396 [==>...........................] - ETA: 18:16 - loss: 0.63 - ETA: 6:26 - loss: 0.7965 - ETA: 3:23 - loss: 0.767 - ETA: 2:40 - loss: 0.742 - ETA: 2:03 - loss: 0.676 - ETA: 1:48 - loss: 0.647 - ETA: 1:31 - loss: 0.600 - ETA: 1:24 - loss: 0.579 - ETA: 1:19 - loss: 0.556 - ETA: 1:16 - loss: 0.540 - ETA: 1:12 - loss: 0.520 - ETA: 1:06 - loss: 0.494 - ETA: 1:01 - loss: 0.469 - ETA: 59s - loss: 0.453 - ETA: 56s - loss: 0.43 - ETA: 54s - loss: 0.42 - ETA: 51s - loss: 0.40 - ETA: 50s - loss: 0.39 - ETA: 50s - loss: 0.39 - ETA: 48s - loss: 0.38 - ETA: 46s - loss: 0.37 - ETA: 45s - loss: 0.36 - ETA: 44s - loss: 0.35 - ETA: 43s - loss: 0.34 - ETA: 42s - loss: 0.34 - ETA: 41s - loss: 0.33 - ETA: 40s - loss: 0.32 - ETA: 39s - loss: 0.31 - ETA: 39s - loss: 0.31 - ETA: 38s - loss: 0.30 - ETA: 38s - loss: 0.30 - ETA: 37s - loss: 0.29 - ET

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()

### Attentive

In [8]:
# due to the model architecture, we need to transform the output shape and type
train_attentive_decoder_target_data = list(np.swapaxes(train_decoder_target_data, 0, 1))
valid_attentive_decoder_target_data = list(np.swapaxes(valid_decoder_target_data, 0, 1))

In [9]:
model_builder = AttentiveSeq2Seq(Tx, Ty, encoder_input_dim, decoder_input_dim,
                                decoder_output_dim)
tuner = RandomSearch(model_builder,
                     objective='val_loss',
                     max_trials=max_trials,
                     executions_per_trial=executions_per_trial,
                     directory='local-keras-tuner', 
                     project_name='attentive')

In [ ]:
tuner.search(x=[train_encoder_input_data, 
                train_decoder_input_data], 
             y=train_attentive_decoder_target_data,
             validation_data=([
                valid_encoder_input_data,
                valid_decoder_input_data],
                valid_attentive_decoder_target_data),
             batch_size=batch_size,
             epochs=epochs,
             verbose=0,
             callbacks=[EarlyStopping(monitor='val_loss', 
                                      patience=patience, 
                                      verbose=1),
                        LossPrintingCallback(Ty)])

In [ ]:
tuner.results_summary()